In [1]:
import chess
import chess.pgn
import pandas as pd
import requests

In [2]:
a_openings = pd.read_csv("https://raw.githubusercontent.com/lichess-org/chess-openings/master/a.tsv", sep="\t")
b_openings = pd.read_csv("https://raw.githubusercontent.com/lichess-org/chess-openings/master/b.tsv", sep="\t")
c_openings = pd.read_csv("https://raw.githubusercontent.com/lichess-org/chess-openings/master/c.tsv", sep="\t")
d_openings = pd.read_csv("https://raw.githubusercontent.com/lichess-org/chess-openings/master/d.tsv", sep="\t")
e_openings = pd.read_csv("https://raw.githubusercontent.com/lichess-org/chess-openings/master/e.tsv", sep="\t")
openings = pd.concat([a_openings, b_openings, c_openings, d_openings, e_openings])


In [3]:
openings

,eco,name,pgn
0,A00,Amar Gambit,1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4
1,A00,Amar Opening,1. Nh3
2,A00,Amar Opening: Gent Gambit,1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...
3,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4
4,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6
...,...,...,...
285,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
286,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
287,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
288,E99,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...


In [4]:
def generate_board_from_opening(opening):
    board = chess.Board()
    for move in opening:
        board.push_san(move)
    return board

In [5]:
def convert_opening_to_san(opening):
    opening = opening["pgn"]
    opening = opening.split(" ")
    opening = [move for move in opening if not move.split(".")[0].isdigit()]
    return opening

In [6]:
opening = openings[openings["name"] == "London System"].iloc[0]
convert_opening_to_san(opening)

['d4', 'Nf6', 'Nf3', 'g6', 'Bf4']

In [7]:
opening = openings[openings["name"] == "London System"].iloc[0]
# remove digits
board = generate_board_from_opening(convert_opening_to_san(opening))
#game = chess.pgn.Game().from_board(board)
game = chess.pgn.Game.from_board(board)

In [9]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver("neo4j+s://bad3437e.databases.neo4j.io:7687", auth=("neo4j", "redacted"))
def execute_query(query):
    with driver.session() as session:
        return session.run(query)

In [10]:
def clean_string(string):
    string = string.replace('"', '\\"')
    string = string.replace("'", "\\'")
    string = string.replace("?","")
    return string

In [11]:
for _, opening in openings.iterrows():
    board = generate_board_from_opening(convert_opening_to_san(opening))
    game = chess.pgn.Game.from_board(board)
    query = f"CREATE (n:Opening {{name: \"{opening['name']}\", eco: \"{opening['eco']}\", fen: \"{board.fen()}\", pgn: \"{opening['pgn']}\"}})"
    execute_query(query)